|평가문항	|상세기준|
|:-|:-|
|1. 모델과 데이터를 정상적으로 불러오고, 작동하는 것을 확인하였다.	|klue/bert-base를 NSMC 데이터셋으로 fine-tuning 하여, 모델이 정상적으로 작동하는 것을 확인하였다.|
|2. Preprocessing을 개선하고, fine-tuning을 통해 모델의 성능을 개선시켰다.	|Validation accuracy를 90% 이상으로 개선하였다.|
|3. 모델 학습에 Bucketing을 성공적으로 적용하고, 그 결과를 비교분석하였다.	|Bucketing task을 수행하여 fine-tuning 시 연산 속도와 모델 성능 간의 trade-off 관계가 발생하는지 여부를 확인하고, 분석한 결과를 제시하였다.|

STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성  
---
데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!  

In [1]:
#라이브러리 버전 확인
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


In [2]:
# nsmc 데이터셋 다운로드
import datasets
from datasets import load_dataset

nsmc_dataset = load_dataset('nsmc')
print(nsmc_dataset)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


NSMC dataset
- Dataset dictionary 안에 train dataset, test dataset으로 구성
- 각 dataset은 'id', 'document', 'label'로 구성되어 있음.

In [3]:
# train datasets의 각 컬럼에 해당하는 몇 가지 요소들을 확인하며 제대로 짝지어져있는지 확인
train = nsmc_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [4]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




긍정적인 평가는 1, 부정적인 평가는 0으로 라벨링되어 있음.

STEP 2. klue/bert-base model 및 tokenizer 불러오기  
---

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [6]:
print(nsmc_dataset['train'].features)

{'id': Value(dtype='string', id=None), 'document': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['negative', 'positive'], names_file=None, id=None)}


STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기  
---

In [7]:
# 데이터셋을 모델이 처리할 수 있는 형식으로 전처리합니다.
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

In [8]:
# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = nsmc_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [9]:
# train & test split
hf_train_dataset = tokenized_datasets['train']
hf_test_dataset = tokenized_datasets['test']

In [10]:
print(len(hf_train_dataset))
print(len(hf_test_dataset))

150000
50000


In [11]:
# train & test split 10%만 샘플링
hf_train_10 = tokenized_datasets['train'].shuffle(seed=42).select(range(15000))
hf_test_10 = tokenized_datasets['test'].shuffle(seed=42).select(range(5000))

In [12]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01                        # weight decay
)

In [13]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    # 예측값 중 가장 높은 확률을 가진 클래스를 선택
    predictions = np.argmax(predictions, axis=1)
    # 실제 레이블과 예측값을 비교하여 정확도를 계산
    return metric.compute(predictions=predictions, references = labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [14]:
trainer = Trainer(
    model=model,                         # 앞서 정의한 모델 인스턴스
    args=training_arguments,             # TrainingArguments 인스턴스
    train_dataset=hf_train_10,      # 훈련 데이터셋
    eval_dataset=hf_test_10,        # 검증 데이터셋, 별도로 정의한 검증 데이터셋이 있으면 그것을 사용하세요.
    compute_metrics=compute_metrics,     # 성능 평가를 위한 메트릭 계산 함수
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.335000,0.313898,0.877200
2,0.216600,0.450585,0.874800
3,0.126100,0.588901,0.880000


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2

TrainOutput(global_step=5625, training_loss=0.2501371673583984, metrics={'train_runtime': 4967.0742, 'train_samples_per_second': 9.06, 'train_steps_per_second': 1.132, 'total_flos': 1.18399974912e+16, 'train_loss': 0.2501371673583984, 'epoch': 3.0})

In [15]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.5949050188064575,
 'eval_accuracy': 0.87902,
 'eval_runtime': 1766.999,
 'eval_samples_per_second': 28.297,
 'eval_steps_per_second': 3.537,
 'epoch': 3.0}

정확도 88%

STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기  
---
데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.  


In [29]:
import re

# NSMC 데이터셋 로드
dataset = load_dataset('nsmc')

# 정규 표현식을 사용하여 한글과 띄어쓰기를 제외한 모든 문자 제거 함수 정의
def remove_non_korean(text):
    return re.sub(r'[^가-힣\s]', '', text)

# 데이터셋의 'document' 필드에 전처리 함수 적용
def preprocess_dataset(example):
    example['document'] = remove_non_korean(example['document'])
    return example

# 훈련 및 테스트 데이터셋에 전처리 함수 적용
preprocessed_dataset = dataset.map(preprocess_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-7de5a5cea8f6e64d.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-a462fab3c6f8b9b5.arrow


In [30]:
# 전처리된 데이터 중 일부 출력해 확인
print(preprocessed_dataset['train'][0]['document'])
print(preprocessed_dataset['train'][1]['document'])

아 더빙 진짜 짜증나네요 목소리
흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나


[링크](https://huggingface.co/sangrimlee/bert-base-multilingual-cased-nsmc)를 참고하여 모델 변경

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("sangrimlee/bert-base-multilingual-cased-nsmc")
model = AutoModelForSequenceClassification.from_pretrained("sangrimlee/bert-base-multilingual-cased-nsmc", num_labels=2)

loading configuration file https://huggingface.co/sangrimlee/bert-base-multilingual-cased-nsmc/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/8dfd196dc647eeecc3a02e5fdcba2371f264501c2935144aabc462888716a112.40a5c8401f1996599a7dae4e5c50067a9c93277a116a5dd33536d29bd041b1a0
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": null,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attentio

In [32]:
# 데이터셋을 모델이 처리할 수 있는 형식으로 전처리합니다.
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

In [33]:
# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = preprocessed_dataset.map(transform, batched=True)

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-cf4c0b31a4f78dde.arrow


  0%|          | 0/50 [00:00<?, ?ba/s]

In [34]:
# train & test split
hf_train_dataset = tokenized_datasets['train']
hf_test_dataset = tokenized_datasets['test']

In [35]:
print(len(hf_train_dataset))
print(len(hf_test_dataset))

150000
50000


In [36]:
# train & test split 10%만 샘플링
hf_train_10 = tokenized_datasets['train'].shuffle(seed=42).select(range(15000))
hf_test_10 = tokenized_datasets['test'].shuffle(seed=42).select(range(5000))

Loading cached shuffled indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-49429d2e47f86562.arrow


In [37]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    # 예측값 중 가장 높은 확률을 가진 클래스를 선택
    predictions = np.argmax(predictions, axis=1)
    # 실제 레이블과 예측값을 비교하여 정확도를 계산
    return metric.compute(predictions=predictions, references = labels)

In [39]:
trainer = Trainer(
    model=model,                         # 앞서 정의한 모델 인스턴스
    args=training_arguments,             # TrainingArguments 인스턴스
    train_dataset=hf_train_10,      # 훈련 데이터셋
    eval_dataset=hf_test_10,        # 검증 데이터셋, 별도로 정의한 검증 데이터셋이 있으면 그것을 사용하세요.
    compute_metrics=compute_metrics,     # 성능 평가를 위한 메트릭 계산 함수
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.402700,0.383188,0.835000
2,0.302100,0.599115,0.837800
3,0.187700,0.747193,0.841800


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2

TrainOutput(global_step=5625, training_loss=0.3052540863037109, metrics={'train_runtime': 5293.906, 'train_samples_per_second': 8.5, 'train_steps_per_second': 1.063, 'total_flos': 1.18399974912e+16, 'train_loss': 0.3052540863037109, 'epoch': 3.0})

In [40]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.7426859140396118,
 'eval_accuracy': 0.8439,
 'eval_runtime': 1763.2928,
 'eval_samples_per_second': 28.356,
 'eval_steps_per_second': 3.545,
 'epoch': 3.0}

정확도 88 -> 84 하락. 전처리는 동일하게 하고, 모델을 다시 KLUE로 바꿔서 테스트.

In [41]:
import re

# NSMC 데이터셋 로드
dataset = load_dataset('nsmc')

# 정규 표현식을 사용하여 한글과 띄어쓰기를 제외한 모든 문자 제거 함수 정의
def remove_non_korean(text):
    return re.sub(r'[^가-힣\s]', '', text)

# 데이터셋의 'document' 필드에 전처리 함수 적용
def preprocess_dataset(example):
    example['document'] = remove_non_korean(example['document'])
    return example

# 훈련 및 테스트 데이터셋에 전처리 함수 적용
preprocessed_dataset = dataset.map(preprocess_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-7de5a5cea8f6e64d.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-a462fab3c6f8b9b5.arrow


In [42]:
# 전처리된 데이터 중 일부 출력해 확인
print(preprocessed_dataset['train'][0]['document'])
print(preprocessed_dataset['train'][1]['document'])

아 더빙 진짜 짜증나네요 목소리
흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나


In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/klue/bert-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/1a36e69d48a0

In [44]:
# 데이터셋을 모델이 처리할 수 있는 형식으로 전처리합니다.
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

In [45]:
# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = preprocessed_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [46]:
# train & test split
hf_train_dataset = tokenized_datasets['train']
hf_test_dataset = tokenized_datasets['test']

In [47]:
print(len(hf_train_dataset))
print(len(hf_test_dataset))

150000
50000


In [48]:
# train & test split 10%만 샘플링
hf_train_10 = tokenized_datasets['train'].shuffle(seed=42).select(range(15000))
hf_test_10 = tokenized_datasets['test'].shuffle(seed=42).select(range(5000))

In [49]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [50]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    # 예측값 중 가장 높은 확률을 가진 클래스를 선택
    predictions = np.argmax(predictions, axis=1)
    # 실제 레이블과 예측값을 비교하여 정확도를 계산
    return metric.compute(predictions=predictions, references = labels)

In [51]:
trainer = Trainer(
    model=model,                         # 앞서 정의한 모델 인스턴스
    args=training_arguments,             # TrainingArguments 인스턴스
    train_dataset=hf_train_10,      # 훈련 데이터셋
    eval_dataset=hf_test_10,        # 검증 데이터셋, 별도로 정의한 검증 데이터셋이 있으면 그것을 사용하세요.
    compute_metrics=compute_metrics,     # 성능 평가를 위한 메트릭 계산 함수
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.356400,0.326528,0.871200
2,0.265400,0.447977,0.871800
3,0.140400,0.585166,0.873200


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2

TrainOutput(global_step=5625, training_loss=0.2701903527153863, metrics={'train_runtime': 5029.2468, 'train_samples_per_second': 8.948, 'train_steps_per_second': 1.118, 'total_flos': 1.18399974912e+16, 'train_loss': 0.2701903527153863, 'epoch': 3.0})

In [52]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.5886006951332092,
 'eval_accuracy': 0.86882,
 'eval_runtime': 1765.81,
 'eval_samples_per_second': 28.316,
 'eval_steps_per_second': 3.539,
 'epoch': 3.0}

88 -> 84 -> 87%.  
klue-bert 대신 다른 모델 사용해보기

In [97]:
del model
del tokenizer

NameError: name 'model' is not defined

In [98]:
import re

# NSMC 데이터셋 로드
dataset = load_dataset('nsmc')

# 정규 표현식을 사용하여 한글과 띄어쓰기를 제외한 모든 문자 제거 함수 정의
def remove_non_korean(text):
    return re.sub(r'[^가-힣\s]', '', text)

# 데이터셋의 'document' 필드에 전처리 함수 적용
def preprocess_dataset(example):
    example['document'] = remove_non_korean(example['document'])
    return example

# 훈련 및 테스트 데이터셋에 전처리 함수 적용
preprocessed_dataset = dataset.map(preprocess_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-7de5a5cea8f6e64d.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-a462fab3c6f8b9b5.arrow


In [99]:
# 전처리된 데이터 중 일부 출력해 확인
print(preprocessed_dataset['train'][0]['document'])
print(preprocessed_dataset['train'][1]['document'])

아 더빙 진짜 짜증나네요 목소리
흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나


In [100]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=2)

https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /aiffel/.cache/huggingface/transformers/tmpdgcrumb3


Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

storing https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json in cache at /aiffel/.cache/huggingface/transformers/9a9f77abeddd1bbd8de28608e78dd3604287ad91abd4796cd25ad936715b7640.5b0ba083b234382bb4c99ee0c9f4fca4cadaa053dd17c32dabfe0de2f629af1f
creating metadata file for /aiffel/.cache/huggingface/transformers/9a9f77abeddd1bbd8de28608e78dd3604287ad91abd4796cd25ad936715b7640.5b0ba083b234382bb4c99ee0c9f4fca4cadaa053dd17c32dabfe0de2f629af1f
https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /aiffel/.cache/huggingface/transformers/tmp5koqhl4l


Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

storing https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt in cache at /aiffel/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
creating metadata file for /aiffel/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /aiffel/.cache/huggingface/transformers/tmp6iojone1


...

storing https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json in cache at /aiffel/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
creating metadata file for /aiffel/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /aiffel/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading 

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

storing https://huggingface.co/klue/roberta-base/resolve/main/config.json in cache at /aiffel/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
creating metadata file for /aiffel/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_pr

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

storing https://huggingface.co/klue/roberta-base/resolve/main/pytorch_model.bin in cache at /aiffel/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320
creating metadata file for /aiffel/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320
loading weights file https://huggingface.co/klue/roberta-base/resolve/main/pytorch_model.bin from cache at /aiffel/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320
Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bi

In [101]:
# 데이터셋을 모델이 처리할 수 있는 형식으로 전처리합니다.
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

In [102]:
# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = preprocessed_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [103]:
# train & test split
hf_train_dataset = tokenized_datasets['train']
hf_test_dataset = tokenized_datasets['test']

In [104]:
print(len(hf_train_dataset))
print(len(hf_test_dataset))

150000
50000


In [105]:
# train & test split 10%만 샘플링
hf_train_10 = tokenized_datasets['train'].shuffle(seed=42).select(range(15000))
hf_test_10 = tokenized_datasets['test'].shuffle(seed=42).select(range(5000))

In [106]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [107]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    # 예측값 중 가장 높은 확률을 가진 클래스를 선택
    predictions = np.argmax(predictions, axis=1)
    # 실제 레이블과 예측값을 비교하여 정확도를 계산
    return metric.compute(predictions=predictions, references = labels)

In [108]:
trainer = Trainer(
    model=model,                         # 앞서 정의한 모델 인스턴스
    args=training_arguments,             # TrainingArguments 인스턴스
    train_dataset=hf_train_10,      # 훈련 데이터셋
    eval_dataset=hf_test_10,        # 검증 데이터셋, 별도로 정의한 검증 데이터셋이 있으면 그것을 사용하세요.
    compute_metrics=compute_metrics,     # 성능 평가를 위한 메트릭 계산 함수
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.384400,0.342800,0.859600
2,0.306000,0.441062,0.871000
3,0.180300,0.540630,0.873200


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoin

TrainOutput(global_step=5625, training_loss=0.3058576107449002, metrics={'train_runtime': 5036.7885, 'train_samples_per_second': 8.934, 'train_steps_per_second': 1.117, 'total_flos': 1.18399974912e+16, 'train_loss': 0.3058576107449002, 'epoch': 3.0})

In [109]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.5392884612083435,
 'eval_accuracy': 0.8712,
 'eval_runtime': 1549.4886,
 'eval_samples_per_second': 32.269,
 'eval_steps_per_second': 4.034,
 'epoch': 3.0}

동일한 전처리 작업을 진행했을 때에는 klue/roberta-base의 성능이 가장 좋았다. (다 비슷하긴 하지만...)  
좀 더 전처리해보자.

In [110]:
del model
del tokenizer

In [111]:
from konlpy.tag import Okt

# KoNLPy의 Okt 객체 초기화
okt = Okt()

# NSMC 데이터셋 로드
dataset = load_dataset('nsmc')

# 불용어 리스트 정의
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

# 전처리 함수 정의
def preprocess_text(text):
    # 한글과 띄어쓰기를 제외한 모든 문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    # 숫자 제거
    text = re.sub(r'\d+', '', text)
    # 어근 추출 및 불용어 제거
    tokenized = okt.morphs(text, stem=True)  # 어근 추출
    tokenized = [word for word in tokenized if not word in stopwords]  # 불용어 제거
    return ' '.join(tokenized)

# 데이터셋 전처리 적용 함수
def preprocess_dataset(example):
    example['document'] = preprocess_text(example['document'])
    return example

# 훈련 및 테스트 데이터셋에 전처리 함수 적용
preprocessed_dataset = dataset.map(preprocess_dataset)

# 전처리된 데이터 중 일부 출력해 확인
print(preprocessed_dataset['train'][0]['document'])
print(preprocessed_dataset['train'][1]['document'])

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

Parameter 'function'=<function preprocess_dataset at 0x7f288636f4c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/150000 [00:00<?, ?ex/s]

  0%|          | 0/50000 [00:00<?, ?ex/s]

아 더빙 진짜 짜증나다 목소리
흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다


In [112]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=2)

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /aiffel/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at /aiffel/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json from 

In [113]:
# 데이터셋을 모델이 처리할 수 있는 형식으로 전처리합니다.
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

In [114]:
# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = preprocessed_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [115]:
# train & test split
hf_train_dataset = tokenized_datasets['train']
hf_test_dataset = tokenized_datasets['test']

In [116]:
print(len(hf_train_dataset))
print(len(hf_test_dataset))

150000
50000


In [117]:
# train & test split 10%만 샘플링
hf_train_10 = tokenized_datasets['train'].shuffle(seed=42).select(range(15000))
hf_test_10 = tokenized_datasets['test'].shuffle(seed=42).select(range(5000))

In [118]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [119]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    # 예측값 중 가장 높은 확률을 가진 클래스를 선택
    predictions = np.argmax(predictions, axis=1)
    # 실제 레이블과 예측값을 비교하여 정확도를 계산
    return metric.compute(predictions=predictions, references = labels)

In [120]:
trainer = Trainer(
    model=model,                         # 앞서 정의한 모델 인스턴스
    args=training_arguments,             # TrainingArguments 인스턴스
    train_dataset=hf_train_10,      # 훈련 데이터셋
    eval_dataset=hf_test_10,        # 검증 데이터셋, 별도로 정의한 검증 데이터셋이 있으면 그것을 사용하세요.
    compute_metrics=compute_metrics,     # 성능 평가를 위한 메트릭 계산 함수
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.416900,0.386791,0.841600
2,0.321600,0.465901,0.850800
3,0.223600,0.581474,0.853800


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoin

TrainOutput(global_step=5625, training_loss=0.33857879028320315, metrics={'train_runtime': 5042.8082, 'train_samples_per_second': 8.924, 'train_steps_per_second': 1.115, 'total_flos': 1.18399974912e+16, 'train_loss': 0.33857879028320315, 'epoch': 3.0})

In [121]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.5909972190856934,
 'eval_accuracy': 0.8512,
 'eval_runtime': 1545.1997,
 'eval_samples_per_second': 32.358,
 'eval_steps_per_second': 4.045,
 'epoch': 3.0}

전처리를 할수록 성능이 떨어짐...모델은 바꾸고 전처리는 최소화, 샘플 수를 늘려서 마지막 테스트.

In [134]:
del model
del tokenizer

In [135]:
#라이브러리 버전 확인
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


In [136]:
# nsmc 데이터셋 다운로드
import datasets
from datasets import load_dataset

nsmc_dataset = load_dataset('nsmc')
print(nsmc_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [137]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=2)

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /aiffel/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at /aiffel/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json from 

In [138]:
print(nsmc_dataset['train'].features)

{'id': Value(dtype='string', id=None), 'document': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['negative', 'positive'], names_file=None, id=None)}


In [139]:
# 데이터셋을 모델이 처리할 수 있는 형식으로 전처리합니다.
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

In [140]:
# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = nsmc_dataset.map(transform, batched=True)

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-d12e978f9c0192a7.arrow


  0%|          | 0/50 [00:00<?, ?ba/s]

In [141]:
# train & test split
hf_train_dataset = tokenized_datasets['train']
hf_test_dataset = tokenized_datasets['test']

In [142]:
print(len(hf_train_dataset))
print(len(hf_test_dataset))

150000
50000


In [143]:
# train & test split 20%만 샘플링
hf_train_20 = tokenized_datasets['train'].shuffle(seed=42).select(range(30000))
hf_test_20 = tokenized_datasets['test'].shuffle(seed=42).select(range(10000))

Loading cached shuffled indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-6568446119b44cb0.arrow


In [144]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 5,                     # train 시킬 총 epochs
    weight_decay = 0.01                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [145]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    # 예측값 중 가장 높은 확률을 가진 클래스를 선택
    predictions = np.argmax(predictions, axis=1)
    # 실제 레이블과 예측값을 비교하여 정확도를 계산
    return metric.compute(predictions=predictions, references = labels)

In [146]:
trainer = Trainer(
    model=model,                         # 앞서 정의한 모델 인스턴스
    args=training_arguments,             # TrainingArguments 인스턴스
    train_dataset=hf_train_20,      # 훈련 데이터셋
    eval_dataset=hf_test_20,        # 검증 데이터셋, 별도로 정의한 검증 데이터셋이 있으면 그것을 사용하세요.
    compute_metrics=compute_metrics,     # 성능 평가를 위한 메트릭 계산 함수
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 30000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18750


Epoch,Training Loss,Validation Loss,Accuracy
1,0.363000,0.323244,0.878900
2,0.307300,0.419268,0.876800
3,0.233300,0.472737,0.891000
4,0.149200,0.581696,0.879400
5,0.086600,0.615328,0.890800


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoi

TrainOutput(global_step=18750, training_loss=0.22518865641276042, metrics={'train_runtime': 16472.5261, 'train_samples_per_second': 9.106, 'train_steps_per_second': 1.138, 'total_flos': 3.9466658304e+16, 'train_loss': 0.22518865641276042, 'epoch': 5.0})

In [147]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 8


{'eval_loss': 0.6351636052131653,
 'eval_accuracy': 0.88646,
 'eval_runtime': 1546.5869,
 'eval_samples_per_second': 32.329,
 'eval_steps_per_second': 4.041,
 'epoch': 5.0}

90%까지는 달성하지 못했지만, 최대 약 89%까지 달성함.

STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교  
---
- 아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.  
    - Data Collator
    - Trainer.TrainingArguments 의 group_by_length   
    
STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.

In [148]:
del model
del tokenizer

In [149]:
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import numpy as np

# NSMC 데이터셋 로드
nsmc_dataset = load_dataset('nsmc')

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=2)

# 데이터셋 전처리 함수
def transform(data):
    return tokenizer(
        data['document'], 
        truncation=True, # 긴 문장을 자르기 위해
        padding='max_length', # 모든 문장을 동일한 길이로 패딩
        return_token_type_ids=False, # NSMC는 단일 문장이므로 token_type_ids는 필요 없음
    )

# 데이터셋 한번에 토크나이징할 수 있는 Map
tokenized_datasets = nsmc_dataset.map(transform, batched=True)

# 데이터셋 분할, 20%만 샘플링
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(30000))
test_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(10000))

# 동적 패딩을 위한 데이터 콜레이터 정의
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

# 학습 파라미터 설정
output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_args = TrainingArguments(
    output_dir='./results',             # 결과가 저장될 경로
    num_train_epochs=5,                 # 총 학습 에포크
    per_device_train_batch_size=8,      # 디바이스 당 훈련 배치 크기
    per_device_eval_batch_size=8,       # 디바이스 당 평가 배치 크기
    warmup_steps=500,                   # 웜업 스텝 수
    weight_decay=0.01,                  # 가중치 감소
    logging_dir='./logs',               # 로깅이 저장될 경로
    evaluation_strategy="epoch",        # 에포크마다 평가
    save_strategy="epoch",              # 에포크마다 모델 저장
    load_best_model_at_end=True,        # 학습 종료 시 가장 좋은 모델을 로드
    metric_for_best_model="accuracy",   # 최고 모델을 결정하기 위한 메트릭
    group_by_length=True                # 유사한 길이의 샘플을 그룹화
)

# 정확도 계산 함수
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 모델 학습
trainer.train()

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /aiffel/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at /aiffel/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json from 

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 30000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18750


Epoch,Training Loss,Validation Loss,Accuracy
1,0.474500,0.434299,0.841500
2,0.604700,0.523179,0.755100
3,0.520200,0.536503,0.777500
4,0.697000,0.698553,0.502900
5,0.380000,0.443468,0.834500


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-3750
Configuration saved in ./results/checkpoint-3750/config.json
Model weights saved in ./results/checkpoint-3750/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-7500
Configuration saved in ./results/checkpoint-7500/config.json
Model weights saved in ./results/checkpoint-7500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
****

TrainOutput(global_step=18750, training_loss=0.4983467692057292, metrics={'train_runtime': 15880.7852, 'train_samples_per_second': 9.445, 'train_steps_per_second': 1.181, 'total_flos': 3.9466658304e+16, 'train_loss': 0.4983467692057292, 'epoch': 5.0})

In [150]:
# 모델 평가
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8


{'eval_loss': 0.4342985451221466,
 'eval_accuracy': 0.8415,
 'eval_runtime': 312.1579,
 'eval_samples_per_second': 32.035,
 'eval_steps_per_second': 4.004,
 'epoch': 5.0}

bucketing과 dynamic padding을 적용하기 전의 결과는 다음과 같다.  
```
{'eval_loss': 0.6351636052131653,
 'eval_accuracy': 0.88646,
 'eval_runtime': 1546.5869,
 'eval_samples_per_second': 32.329,
 'eval_steps_per_second': 4.041,
 'epoch': 5.0}
```
bucketing과 dynamic padding을 적용한 후의 정확도가 더 낮지만 loss가 더 작고 평가 시간이 1/5 정도로 단축되었다.  
- 모델 선택 시 정확도, 손실, 평가 시간 등 여러 가지 지표를 고려해야 함
- 특정 상황이나 목표에 따라 최적의 모델을 선택하는 전략이 달라질 수 있음
    - 시간 효율성이 중요한 경우 두 번째 모델을, 정확도가 중요하다면 첫 번째 모델이 더 적합함

# 회고
결국 90%까지 올리진 못했네요...아직도 트랜스포머를 잘 이해하지는 못했지만 뭔가 이런 식으로도 딥러닝을 할 수 있다는 건 조금은 이해한 것 같습니다.  
90% 이상 올리신 분들은 무슨 모델 쓰셨나 궁금하네요ㅎㅎ